In [1]:
# set run environment (local/colab)
# if colab install required packages and set appropriate root_path
import os
from pathlib import Path
if os.getenv("COLAB_RELEASE_TAG"):
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = Path('/content/drive/Othercomputers/My computer/EQILLM/')
    !pip install -r '/content/drive/Othercomputers/My computer/EQILLM/requirements.txt'
    !pip install transformers[torch]
    !pip install accelerate -U
    !pip install datasets
    !pip install torchinfo
    import sys
    sys.path.append(root_path)
    #ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U
else:
    colab = False
    root_path = Path('')


import itertools
import pandas as pd
import openai
import datetime
import os
import csv
import matplotlib.pyplot as plt
import random

from dotenv import load_dotenv, dotenv_values
from tqdm.notebook import tqdm_notebook
from eqillm import finetune, get_log_for_val, validate, val_metrics, yeelight_eow_notification, param_combinations, load_PolarIs, df_to_ds


dotenv_config = dotenv_values(root_path / '.env')
yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if 'YEELIGHT_NOTIFY' in dotenv_config else False

In [2]:
def get_response(text, parameters, client):
    completion = client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=create_messages(text),
        **parameters
    )
    return completion.choices[0].message.content

def looped_prompt(texts, labels, parameters, client):
    
    model_name = client.models.list().data[0].dict()['id']
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    
    prompt_log_path = root_path / 'output/prompting/prompt_log.csv'
    response_log_path = root_path / f'output/prompting/responses/{timestamp}.csv'

    # add row to prompt log
    with open(prompt_log_path, mode='a+', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if not os.path.exists(prompt_log_path):
            writer.writerow(['model', 'timestamp', 'message', 'params'])
        writer.writerow([model_name, timestamp, create_messages(), parameters])

    with open(response_log_path, mode='w', newline='', encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file,  delimiter='|')
        writer.writerow(['text', 'label', 'timestamp', 'response'])

        for i, (txt, label) in tqdm_notebook(enumerate(zip(texts, labels))): # iterate through each text value of first df column
            response = get_response(txt, par, client)
            writer.writerow([txt, label, timestamp, response])
            csv_file.flush()

In [3]:
par = {'temperature': 0.7,
       'max_tokens': 50,
       'stop': ['Pathos'],
       }



data_path = root_path / 'data/PolarIs-Pathos.xlsx'
polarIs_df = load_PolarIs(data_path)

dataset, target_map, reversed_target_map = df_to_ds(polarIs_df, split=(0.9, 0.1), binary=True, balanced=True)

shuffle = True
if shuffle:
    random.seed(42)
    randomized = list(zip(dataset['train']['text'], [reversed_target_map[i] for i in dataset['train']['label']]))
    random.shuffle(randomized)
    val_txts = [i[0] for i in randomized]
    val_labels = [i[1] for i in randomized]
else:
    val_txts, val_labels = dataset['train']['text'], [reversed_target_map[i] for i in dataset['train']['label']]

def create_messages(text='_'):
    return [
        {
            'role': 'system',
             'content': """You are an analytical tool designed to identify rhetorical pathos in text. Your task is to analyze if the speaker's intention is to elicit specific emotional response for rhetorical purposes. The only responses to consider are: 'Pathos', 'No Pathos'. Do not infer emotions not explicitly stated in the text, and do not identify emotions outside of two specified. """
        },
        {
            "role": "assistant",
            "content": "Based on the analysis, is there an attempt to elicit an emotional reponse? If so, identify only one of: 'Pathos', 'No Pathos'. Do not identify emotions not listed."
        },
        {
            "role": "user",
            "content": f"""Analyze the following text for pathos: {text}"""
        },

    ]

In [4]:
client = openai.OpenAI(base_url=dotenv_config['MODEL_URL'],  api_key=dotenv_config['MODEL_API_KEY'])
looped_prompt(val_txts, val_labels, par, client)

if yeelight_notify:
    yeelight_eow_notification(dotenv_config['YEELIGHT_PORT'])

# import glob